In [ ]:
import arcpy
import os

# 设置工作空间
arcpy.env.workspace = r"D:\ArcGISProjects\workspace\sp2024\MyProject.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# 输入参数
input_feature = "sb_merge_data__MultipartToSi1"  # 输入要处理的面要素类
output_feature = "sb_merge_data__MultipartToSi1_rsult"  # 输出处理后的面要素类
area_threshold = 50  # 面积阈值（平方米）
dlmc_field = "DLMC"  # DLMC字段名


# 获取所有唯一的DLMC值
with arcpy.da.SearchCursor(input_feature, [dlmc_field]) as cursor:
    dlmc_values = sorted(set(row[0] for row in cursor))

print(f"发现 {len(dlmc_values)} 个不同的DLMC类型")

In [ ]:

# 创建第一个临时结果要素类
temp_result = "in_memory/temp_result"
arcpy.CopyFeatures_management(input_feature, temp_result)

In [ ]:

# 为每个DLMC值进行处理
for index, dlmc_value in enumerate(dlmc_values):
    print(f"\n正在处理DLMC {index}/{len(dlmc_values)}: {dlmc_value}")
    # 创建要素图层
    arcpy.MakeFeatureLayer_management(temp_result, "working_layer")
    
    # 选择当前DLMC值的小于阈值面积的图斑
    where_clause = f"Shape_Area < {area_threshold} AND {dlmc_field} = '{dlmc_value}'"
    arcpy.SelectLayerByAttribute_management("working_layer", "NEW_SELECTION", where_clause)
    
    # 获取选择的图斑数量
    selected_count = int(arcpy.GetCount_management("working_layer")[0])
    
    if selected_count > 0:
        print(f"  发现 {selected_count} 个小于 {area_threshold} 平方米的图斑")
        
        # 执行Eliminate操作，结果存储到新的临时要素类
        temp_eliminated = f"in_memory/temp_eliminated_{index}"
        arcpy.management.Eliminate(
            "working_layer", 
            temp_eliminated,
            "LENGTH", 
            f"{dlmc_field} <> '{dlmc_value}'",  # 保证只合并到相同DLMC类型
            None
        )
        
        # 删除上一轮的临时结果，并将新结果复制为temp_result
        arcpy.Delete_management(temp_result)
        arcpy.CopyFeatures_management(temp_eliminated, temp_result)
        arcpy.Delete_management(temp_eliminated)
        
        print("  已完成消除操作")
    else:
        print("  未发现需要处理的小图斑")
    
    # 清理临时图层
    arcpy.Delete_management("working_layer")

# 将最后的临时结果复制到最终输出要素类
arcpy.CopyFeatures_management(temp_result, output_feature)

# 删除最后的临时结果
arcpy.Delete_management(temp_result)

print(f"\n所有DLMC类型的处理已完成。最终结果保存在: {os.path.join(arcpy.env.workspace, output_feature)}")

# # 重建空间索引和统计信息
# arcpy.AddSpatialIndex_management(output_feature)
# arcpy.CalculateStatistics_management(output_feature)

print("空间索引和统计信息已更新。处理全部完成。")